# Loss Functions: Mathematical Foundations & Custom Implementations

## 🎯 **Amazon Applied Scientist Interview Preparation**

This notebook provides **comprehensive coverage** of loss functions essential for ML interviews, with focus on:

### 📚 **What You'll Master:**
1. **Mathematical derivations** of each loss function
2. **Custom implementations** from scratch (no libraries)
3. **Gradient computations** for optimization
4. **When to use** each loss function
5. **Numerical stability** considerations
6. **Business applications** and trade-offs

### 🔥 **Loss Functions Covered:**
- **Regression**: MSE, MAE, Huber, Quantile
- **Classification**: Binary Cross-Entropy, Categorical CE, Focal, Hinge
- **Advanced**: Triplet, Contrastive, Wasserstein, KL Divergence
- **Custom**: Weighted losses, Label smoothing

### ⚠️ **Interview Focus Areas:**
1. ✅ Derive gradients by hand
2. ✅ Implement numerical stability fixes
3. ✅ Explain when each loss is optimal
4. ✅ Connect to business metrics
5. ✅ Handle edge cases and overflow
6. ✅ Compare computational complexity

---
**🚨 Key Interview Insight**: Loss function choice can make or break model performance. Understanding the "why" behind each formula is crucial for Amazon's technical depth rounds.

## 1. Essential Imports and Setup

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Union, Tuple, Optional
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 11

# Mathematical constants
EPSILON = 1e-15  # Numerical stability constant
LOG_EPSILON = 1e-8  # For log operations

print("📦 IMPORTS COMPLETE")
print("🎯 Ready for loss function implementations!")
print(f"📊 NumPy version: {np.__version__}")
print(f"🔢 Numerical epsilon: {EPSILON}")
print(f"📈 Matplotlib backend: {plt.get_backend()}")

# Quick test of numerical stability
print(f"\n🧪 NUMERICAL STABILITY TESTS:")
print(f"   log(EPSILON) = {np.log(EPSILON):.2f}")
print(f"   exp(700) = {np.exp(700) if 700 < 709 else 'overflow!'}")
print(f"   exp(-700) = {np.exp(-700):.2e}")
print("✅ Environment ready for stable implementations!")

## 2. Regression Loss Functions

### 🎯 **Mathematical Foundation**

Regression losses measure the difference between predicted continuous values and true targets. Key considerations:

| **Loss Function** | **Formula** | **Gradient** | **Robustness** | **Use Case** |
|-------------------|-------------|--------------|----------------|---------------|
| **MSE (L2)** | `½(y - ŷ)²` | `-(y - ŷ)` | Sensitive to outliers | Gaussian noise, smooth optimization |
| **MAE (L1)** | `|y - ŷ|` | `sign(ŷ - y)` | Robust to outliers | Heavy-tailed noise, median regression |
| **Huber** | Hybrid L1/L2 | Smooth transition | Balanced robustness | Robust regression with smooth gradients |
| **Quantile** | Asymmetric L1 | Weighted sign | Robust, asymmetric | Risk modeling, confidence intervals |

### 🚨 **Critical Interview Insights:**
- **MSE** penalizes large errors quadratically → sensitive to outliers
- **MAE** treats all errors equally → robust but non-differentiable at 0
- **Huber** combines both → robust with smooth gradients
- **Quantile** enables uncertainty quantification → essential for risk assessment

In [ ]:
class RegressionLosses:
    """
    Custom implementations of regression loss functions.
    
    🎯 INTERVIEW FOCUS: Understand mathematical properties and implementation trade-offs
    """
    
    @staticmethod
    def mse_loss(y_true: np.ndarray, y_pred: np.ndarray, reduction: str = 'mean') -> float:
        """
        Mean Squared Error (L2 Loss)
        
        📚 MATHEMATICAL FOUNDATION:
        L(y, ŷ) = ½(y - ŷ)²
        
        🔍 GRADIENT:
        ∂L/∂ŷ = -(y - ŷ) = (ŷ - y)
        
        ⚠️ PROPERTIES:
        • Differentiable everywhere
        • Convex (single global minimum)
        • Sensitive to outliers (quadratic penalty)
        • Optimal for Gaussian noise
        
        Args:
            y_true: Ground truth values
            y_pred: Predicted values
            reduction: 'mean', 'sum', or 'none'
        """
        diff = y_pred - y_true
        squared_error = 0.5 * diff**2
        
        if reduction == 'mean':
            return np.mean(squared_error)
        elif reduction == 'sum':
            return np.sum(squared_error)
        else:
            return squared_error
    
    @staticmethod
    def mse_gradient(y_true: np.ndarray, y_pred: np.ndarray) -> np.ndarray:
        """
        Gradient of MSE w.r.t. predictions
        
        ∂L/∂ŷ = (ŷ - y)
        """
        return y_pred - y_true
    
    @staticmethod
    def mae_loss(y_true: np.ndarray, y_pred: np.ndarray, reduction: str = 'mean') -> float:
        """
        Mean Absolute Error (L1 Loss)
        
        📚 MATHEMATICAL FOUNDATION:
        L(y, ŷ) = |y - ŷ|
        
        🔍 GRADIENT:
        ∂L/∂ŷ = sign(ŷ - y)  [undefined at ŷ = y]
        
        ⚠️ PROPERTIES:
        • Non-differentiable at ŷ = y
        • Robust to outliers (linear penalty)
        • Optimal for Laplace noise
        • Promotes sparsity
        
        🚨 CAVEAT: Gradient is discontinuous at zero!
        """
        abs_error = np.abs(y_pred - y_true)
        
        if reduction == 'mean':
            return np.mean(abs_error)
        elif reduction == 'sum':
            return np.sum(abs_error)
        else:
            return abs_error
    
    @staticmethod
    def mae_gradient(y_true: np.ndarray, y_pred: np.ndarray, epsilon: float = 1e-8) -> np.ndarray:
        """
        Gradient of MAE w.r.t. predictions (with smoothing for numerical stability)
        
        ∂L/∂ŷ = sign(ŷ - y)
        
        🔧 NUMERICAL FIX: Use smooth approximation near zero
        """
        diff = y_pred - y_true
        # Smooth sign function to avoid undefined gradient at 0
        return np.tanh(diff / epsilon)
    
    @staticmethod
    def huber_loss(y_true: np.ndarray, y_pred: np.ndarray, delta: float = 1.0, reduction: str = 'mean') -> float:
        """
        Huber Loss (Smooth L1 Loss)
        
        📚 MATHEMATICAL FOUNDATION:
        L(y, ŷ) = {
            ½(y - ŷ)²           if |y - ŷ| ≤ δ
            δ|y - ŷ| - ½δ²      if |y - ŷ| > δ
        }
        
        🔍 GRADIENT:
        ∂L/∂ŷ = {
            (ŷ - y)           if |y - ŷ| ≤ δ
            δ·sign(ŷ - y)     if |y - ŷ| > δ
        }
        
        ⚠️ PROPERTIES:
        • Smooth everywhere (differentiable)
        • Quadratic for small errors (MSE-like)
        • Linear for large errors (MAE-like)
        • Balanced robustness
        
        Args:
            delta: Threshold between quadratic and linear regions
        """
        diff = y_pred - y_true
        abs_diff = np.abs(diff)
        
        # Quadratic region: |error| ≤ δ
        quadratic_mask = abs_diff <= delta
        quadratic_loss = 0.5 * diff**2
        
        # Linear region: |error| > δ
        linear_mask = abs_diff > delta
        linear_loss = delta * abs_diff - 0.5 * delta**2
        
        loss = quadratic_mask * quadratic_loss + linear_mask * linear_loss
        
        if reduction == 'mean':
            return np.mean(loss)
        elif reduction == 'sum':
            return np.sum(loss)
        else:
            return loss
    
    @staticmethod
    def huber_gradient(y_true: np.ndarray, y_pred: np.ndarray, delta: float = 1.0) -> np.ndarray:
        """
        Gradient of Huber loss w.r.t. predictions
        """
        diff = y_pred - y_true
        abs_diff = np.abs(diff)
        
        # Quadratic region gradient: (ŷ - y)
        quadratic_mask = abs_diff <= delta
        quadratic_grad = diff
        
        # Linear region gradient: δ·sign(ŷ - y)
        linear_mask = abs_diff > delta
        linear_grad = delta * np.sign(diff)
        
        return quadratic_mask * quadratic_grad + linear_mask * linear_grad

# Test the implementations
print("🧪 TESTING REGRESSION LOSSES")
print("=" * 40)

# Generate test data
np.random.seed(42)
y_true = np.array([1.0, 2.0, 3.0, 100.0])  # Note: 100.0 is an outlier
y_pred = np.array([1.1, 2.2, 2.8, 10.0])   # Outlier prediction is way off

losses = RegressionLosses()

# Calculate losses
mse = losses.mse_loss(y_true, y_pred)
mae = losses.mae_loss(y_true, y_pred)
huber = losses.huber_loss(y_true, y_pred, delta=1.0)

print(f"Test data:")
print(f"  y_true: {y_true}")
print(f"  y_pred: {y_pred}")
print(f"  errors: {y_pred - y_true}")
print(f"\nLoss comparison:")
print(f"  MSE:    {mse:.4f} (sensitive to outlier)")
print(f"  MAE:    {mae:.4f} (robust to outlier)")
print(f"  Huber:  {huber:.4f} (balanced)")

# Test gradients
mse_grad = losses.mse_gradient(y_true, y_pred)
mae_grad = losses.mae_gradient(y_true, y_pred)
huber_grad = losses.huber_gradient(y_true, y_pred)

print(f"\nGradient comparison:")
print(f"  MSE grad:   {mse_grad}")
print(f"  MAE grad:   {mae_grad}")
print(f"  Huber grad: {huber_grad}")

print(f"\n💡 INTERVIEW INSIGHT:")
print(f"   Notice how MSE gradient grows linearly with error size,")
print(f"   while MAE gradient is constant (±1) regardless of error magnitude.")
print(f"   Huber combines both: smooth near zero, clipped for large errors.")

In [ ]:
# Add Quantile Loss to RegressionLosses class
def quantile_loss(y_true: np.ndarray, y_pred: np.ndarray, quantile: float = 0.5, reduction: str = 'mean') -> float:
    """
    Quantile Loss (Pinball Loss)
    
    📚 MATHEMATICAL FOUNDATION:
    L_τ(y, ŷ) = {
        τ(y - ŷ)       if y ≥ ŷ  (under-prediction)
        (τ-1)(y - ŷ)   if y < ŷ  (over-prediction)
    }
    
    🔍 GRADIENT:
    ∂L/∂ŷ = {
        -τ      if y ≥ ŷ
        -(τ-1)  if y < ŷ
    }
    
    ⚠️ PROPERTIES:
    • Asymmetric penalty (different costs for over/under-prediction)
    • τ = 0.5 → MAE (median regression)
    • τ ≠ 0.5 → Biased toward quantile
    • Enables uncertainty quantification
    
    🎯 BUSINESS APPLICATIONS:
    • Risk management (Value at Risk)
    • Inventory optimization
    • Demand forecasting with asymmetric costs
    
    Args:
        quantile: Target quantile τ ∈ (0, 1)
    """
    error = y_true - y_pred
    
    # Asymmetric penalty
    over_pred_mask = error < 0  # y < ŷ (over-prediction)
    under_pred_mask = error >= 0  # y ≥ ŷ (under-prediction)
    
    loss = (under_pred_mask * quantile * error + 
            over_pred_mask * (quantile - 1) * error)
    
    if reduction == 'mean':
        return np.mean(loss)
    elif reduction == 'sum':
        return np.sum(loss)
    else:
        return loss

def quantile_gradient(y_true: np.ndarray, y_pred: np.ndarray, quantile: float = 0.5) -> np.ndarray:
    """Gradient of quantile loss w.r.t. predictions"""
    error = y_true - y_pred
    over_pred_mask = error < 0
    under_pred_mask = error >= 0
    
    return -(under_pred_mask * quantile + over_pred_mask * (quantile - 1))

# Add methods to the class
RegressionLosses.quantile_loss = staticmethod(quantile_loss)
RegressionLosses.quantile_gradient = staticmethod(quantile_gradient)

# Visualize regression losses
def visualize_regression_losses():
    """Visualize how different regression losses behave"""
    errors = np.linspace(-5, 5, 1000)
    y_true = np.zeros_like(errors)
    y_pred = errors  # So error = y_pred - y_true = errors
    
    losses = RegressionLosses()
    
    mse_vals = [losses.mse_loss(np.array([0]), np.array([e]), reduction='none')[0] for e in errors]
    mae_vals = [losses.mae_loss(np.array([0]), np.array([e]), reduction='none')[0] for e in errors]
    huber_vals = [losses.huber_loss(np.array([0]), np.array([e]), delta=1.0, reduction='none')[0] for e in errors]
    quantile_05_vals = [losses.quantile_loss(np.array([0]), np.array([e]), quantile=0.05, reduction='none')[0] for e in errors]
    quantile_95_vals = [losses.quantile_loss(np.array([0]), np.array([e]), quantile=0.95, reduction='none')[0] for e in errors]
    
    plt.figure(figsize=(15, 10))
    
    # Plot 1: Loss functions
    plt.subplot(2, 2, 1)
    plt.plot(errors, mse_vals, 'b-', linewidth=2, label='MSE (L2)')
    plt.plot(errors, mae_vals, 'r-', linewidth=2, label='MAE (L1)')
    plt.plot(errors, huber_vals, 'g-', linewidth=2, label='Huber (δ=1)')
    plt.xlabel('Prediction Error (ŷ - y)')
    plt.ylabel('Loss Value')
    plt.title('Regression Loss Functions\n(MSE vs MAE vs Huber)', fontweight='bold')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.axhline(y=0, color='k', linestyle='-', alpha=0.3)
    plt.axvline(x=0, color='k', linestyle='-', alpha=0.3)
    
    # Plot 2: Gradients
    plt.subplot(2, 2, 2)
    mse_grads = errors  # Gradient of MSE is just the error
    mae_grads = np.sign(errors)  # Gradient of MAE is sign function
    huber_grads = [losses.huber_gradient(np.array([0]), np.array([e]))[0] for e in errors]
    
    plt.plot(errors, mse_grads, 'b-', linewidth=2, label='MSE Gradient')
    plt.plot(errors, mae_grads, 'r-', linewidth=2, label='MAE Gradient')
    plt.plot(errors, huber_grads, 'g-', linewidth=2, label='Huber Gradient')
    plt.xlabel('Prediction Error (ŷ - y)')
    plt.ylabel('Gradient Value')
    plt.title('Loss Function Gradients', fontweight='bold')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.axhline(y=0, color='k', linestyle='-', alpha=0.3)
    plt.axvline(x=0, color='k', linestyle='-', alpha=0.3)
    
    # Plot 3: Quantile losses
    plt.subplot(2, 2, 3)
    plt.plot(errors, quantile_05_vals, 'purple', linewidth=2, label='Quantile 0.05 (5th percentile)')
    plt.plot(errors, mae_vals, 'orange', linewidth=2, label='Quantile 0.5 (median = MAE)')
    plt.plot(errors, quantile_95_vals, 'brown', linewidth=2, label='Quantile 0.95 (95th percentile)')
    plt.xlabel('Prediction Error (ŷ - y)')
    plt.ylabel('Loss Value')
    plt.title('Quantile Loss Functions\n(Asymmetric Penalties)', fontweight='bold')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.axhline(y=0, color='k', linestyle='-', alpha=0.3)
    plt.axvline(x=0, color='k', linestyle='-', alpha=0.3)
    
    # Plot 4: Outlier sensitivity comparison
    plt.subplot(2, 2, 4)
    outlier_errors = np.array([-10, -5, -1, 0, 1, 5, 10])
    mse_outlier = [losses.mse_loss(np.array([0]), np.array([e]), reduction='none')[0] for e in outlier_errors]
    mae_outlier = [losses.mae_loss(np.array([0]), np.array([e]), reduction='none')[0] for e in outlier_errors]
    huber_outlier = [losses.huber_loss(np.array([0]), np.array([e]), delta=1.0, reduction='none')[0] for e in outlier_errors]
    
    x_pos = np.arange(len(outlier_errors))
    width = 0.25
    
    plt.bar(x_pos - width, mse_outlier, width, label='MSE', alpha=0.8)
    plt.bar(x_pos, mae_outlier, width, label='MAE', alpha=0.8)
    plt.bar(x_pos + width, huber_outlier, width, label='Huber', alpha=0.8)
    
    plt.xlabel('Prediction Error')
    plt.ylabel('Loss Value')
    plt.title('Outlier Sensitivity Comparison', fontweight='bold')
    plt.xticks(x_pos, outlier_errors)
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# Run visualization
visualize_regression_losses()

print("📊 VISUALIZATION ANALYSIS:")
print("1. MSE grows quadratically → heavily penalizes outliers")
print("2. MAE grows linearly → robust to outliers")
print("3. Huber combines both → smooth gradients + robustness")
print("4. Quantile losses → asymmetric penalties for risk modeling")
print("\n🎯 INTERVIEW TAKEAWAY:")
print("   Choose loss function based on noise distribution and business requirements!")

# Test quantile loss
print(f"\n🧪 QUANTILE LOSS DEMONSTRATION:")
y_true = np.array([10.0])
y_pred_under = np.array([8.0])  # Under-prediction
y_pred_over = np.array([12.0])  # Over-prediction

for q in [0.1, 0.5, 0.9]:
    under_loss = losses.quantile_loss(y_true, y_pred_under, quantile=q)
    over_loss = losses.quantile_loss(y_true, y_pred_over, quantile=q)
    print(f"   Quantile {q}: Under-pred loss = {under_loss:.3f}, Over-pred loss = {over_loss:.3f}")

print(f"\n💡 Notice: Low quantiles (0.1) penalize under-prediction more heavily")
print(f"           High quantiles (0.9) penalize over-prediction more heavily")

## 3. Classification Loss Functions

### 🎯 **Mathematical Foundation**

Classification losses measure prediction quality for discrete labels. Critical for understanding optimization dynamics:

| **Loss Function** | **Formula** | **Gradient** | **Use Case** | **Key Properties** |
|-------------------|-------------|--------------|---------------|-------------------|
| **Binary Cross-Entropy** | `-[y log(p) + (1-y) log(1-p)]` | `p - y` | Binary classification | Probabilistic, smooth gradients |
| **Categorical Cross-Entropy** | `-Σ y_i log(p_i)` | `p_i - y_i` | Multi-class | One-hot encoded targets |
| **Sparse Cross-Entropy** | `-log(p_target)` | `p_i - δ_i` | Multi-class | Integer targets |
| **Focal Loss** | `-α(1-p)^γ log(p)` | Complex | Imbalanced data | Down-weights easy examples |
| **Hinge Loss** | `max(0, 1 - y·f(x))` | `∂/∂f` | SVM-style | Large margin, sparse |

### 🚨 **Numerical Stability Challenges:**

1. **log(0) = -∞**: Must clip probabilities
2. **exp overflow**: Use log-sum-exp trick
3. **Gradient explosion**: Careful with logits
4. **Class imbalance**: Weight adjustments needed

### 🎯 **Amazon Interview Focus:**
- When to use each loss function
- Numerical stability implementations
- Gradient derivations by hand
- Business impact of loss choice

In [ ]:
class ClassificationLosses:
    """
    Custom implementations of classification loss functions.
    
    🎯 INTERVIEW FOCUS: Numerical stability and gradient derivations
    """
    
    @staticmethod
    def sigmoid(logits: np.ndarray) -> np.ndarray:
        """
        Numerically stable sigmoid function
        
        🚨 CRITICAL: Prevents overflow for large |logits|
        """
        return np.where(logits >= 0,
                        1 / (1 + np.exp(-logits)),
                        np.exp(logits) / (1 + np.exp(logits)))
    
    @staticmethod
    def softmax(logits: np.ndarray, axis: int = -1) -> np.ndarray:
        """
        Numerically stable softmax function
        
        🚨 CRITICAL: Subtracts max to prevent overflow
        
        📚 FORMULA: softmax(x_i) = exp(x_i) / Σ exp(x_j)
        🔧 STABLE: softmax(x_i) = exp(x_i - max(x)) / Σ exp(x_j - max(x))
        """
        # Subtract max for numerical stability
        shifted_logits = logits - np.max(logits, axis=axis, keepdims=True)
        exp_logits = np.exp(shifted_logits)
        return exp_logits / np.sum(exp_logits, axis=axis, keepdims=True)
    
    @staticmethod
    def binary_crossentropy(y_true: np.ndarray, y_pred: np.ndarray, 
                           epsilon: float = EPSILON, reduction: str = 'mean') -> float:
        """
        Binary Cross-Entropy Loss (from probabilities)
        
        📚 MATHEMATICAL FOUNDATION:
        L = -[y log(p) + (1-y) log(1-p)]
        
        🔍 GRADIENT w.r.t. probabilities:
        ∂L/∂p = -(y/p) + (1-y)/(1-p) = (p-y)/[p(1-p)]
        
        🔍 GRADIENT w.r.t. logits (z = log(p/(1-p))):
        ∂L/∂z = p - y  (much cleaner!)
        
        ⚠️ NUMERICAL ISSUES:
        • log(0) = -∞ → clip probabilities
        • p ∈ [ε, 1-ε] prevents overflow
        
        🎯 INTERVIEW INSIGHT:
        Working in logit space (before sigmoid) is more stable!
        """
        # Clip probabilities to prevent log(0)
        y_pred_clipped = np.clip(y_pred, epsilon, 1 - epsilon)
        
        # Binary cross-entropy formula
        loss = -(y_true * np.log(y_pred_clipped) + 
                 (1 - y_true) * np.log(1 - y_pred_clipped))
        
        if reduction == 'mean':
            return np.mean(loss)
        elif reduction == 'sum':
            return np.sum(loss)
        else:
            return loss
    
    @staticmethod
    def binary_crossentropy_with_logits(y_true: np.ndarray, logits: np.ndarray,
                                       reduction: str = 'mean') -> float:
        """
        Binary Cross-Entropy with Logits (PREFERRED!)
        
        📚 MATHEMATICAL FOUNDATION:
        L = max(z, 0) - z·y + log(1 + exp(-|z|))
        
        Where z = logits, y = labels
        
        🔧 NUMERICAL STABILITY:
        For z ≥ 0: log(1 + exp(-z)) = log(1 + exp(-z))
        For z < 0:  log(1 + exp(z)) = z + log(1 + exp(-z))
        
        ⚠️ WHY BETTER:
        • No explicit sigmoid computation
        • No probability clipping needed
        • Numerically stable for all logit values
        • Gradients are well-behaved
        
        🎯 AMAZON INSIGHT: Always prefer logit-based losses!
        """
        # Numerically stable implementation
        # L = max(z, 0) - z*y + log(1 + exp(-|z|))
        
        # Separate positive and negative logits for stability
        positive_logits = np.maximum(logits, 0)
        negative_part = np.maximum(-logits, 0)
        
        loss = (positive_logits - 
                logits * y_true + 
                np.log(1 + np.exp(-negative_part)) + 
                np.where(logits < 0, -logits, 0))
        
        if reduction == 'mean':
            return np.mean(loss)
        elif reduction == 'sum':
            return np.sum(loss)
        else:
            return loss
    
    @staticmethod
    def binary_crossentropy_gradient(y_true: np.ndarray, logits: np.ndarray) -> np.ndarray:
        """
        Gradient of BCE w.r.t. logits
        
        🔍 BEAUTIFUL RESULT: ∂L/∂z = σ(z) - y = p - y
        
        This is why BCE + sigmoid is so popular in neural networks!
        """
        probabilities = ClassificationLosses.sigmoid(logits)
        return probabilities - y_true
    
    @staticmethod
    def categorical_crossentropy(y_true: np.ndarray, y_pred: np.ndarray,
                                epsilon: float = EPSILON, reduction: str = 'mean') -> float:
        """
        Categorical Cross-Entropy Loss (from probabilities)
        
        📚 MATHEMATICAL FOUNDATION:
        L = -Σ y_i log(p_i)
        
        🔍 GRADIENT w.r.t. probabilities:
        ∂L/∂p_i = -y_i / p_i
        
        🔍 GRADIENT w.r.t. logits (before softmax):
        ∂L/∂z_i = p_i - y_i  (clean and simple!)
        
        Args:
            y_true: One-hot encoded labels (n_samples, n_classes)
            y_pred: Predicted probabilities (n_samples, n_classes)
        """
        # Clip probabilities to prevent log(0)
        y_pred_clipped = np.clip(y_pred, epsilon, 1 - epsilon)
        
        # Categorical cross-entropy: sum over classes, mean over samples
        loss = -np.sum(y_true * np.log(y_pred_clipped), axis=1)
        
        if reduction == 'mean':
            return np.mean(loss)
        elif reduction == 'sum':
            return np.sum(loss)
        else:
            return loss
    
    @staticmethod
    def categorical_crossentropy_with_logits(y_true: np.ndarray, logits: np.ndarray,
                                           reduction: str = 'mean') -> float:
        """
        Categorical Cross-Entropy with Logits (PREFERRED!)
        
        📚 MATHEMATICAL FOUNDATION:
        L = -Σ y_i (z_i - log_sum_exp(z))
        L = log_sum_exp(z) - Σ y_i z_i
        
        🔧 NUMERICAL STABILITY: Uses log-sum-exp trick
        """
        # Compute log_sum_exp(logits) for each sample
        max_logits = np.max(logits, axis=1, keepdims=True)
        shifted_logits = logits - max_logits
        log_sum_exp = max_logits.squeeze() + np.log(np.sum(np.exp(shifted_logits), axis=1))
        
        # Compute -Σ y_i z_i for each sample
        target_logits = np.sum(y_true * logits, axis=1)
        
        loss = log_sum_exp - target_logits
        
        if reduction == 'mean':
            return np.mean(loss)
        elif reduction == 'sum':
            return np.sum(loss)
        else:
            return loss
    
    @staticmethod
    def categorical_crossentropy_gradient(y_true: np.ndarray, logits: np.ndarray) -> np.ndarray:
        """
        Gradient of categorical CE w.r.t. logits
        
        🔍 BEAUTIFUL RESULT: ∂L/∂z_i = softmax(z_i) - y_i
        """
        probabilities = ClassificationLosses.softmax(logits)
        return probabilities - y_true
    
    @staticmethod
    def sparse_categorical_crossentropy(y_true: np.ndarray, logits: np.ndarray,
                                      reduction: str = 'mean') -> float:
        """
        Sparse Categorical Cross-Entropy (integer labels)
        
        📚 MATHEMATICAL FOUNDATION:
        Same as categorical CE, but y_true are integers, not one-hot
        
        L = -log(p_target) = -(z_target - log_sum_exp(z))
        
        Args:
            y_true: Integer labels (n_samples,)
            logits: Raw logits (n_samples, n_classes)
        """
        n_samples = logits.shape[0]
        
        # Extract logits for true classes
        target_logits = logits[np.arange(n_samples), y_true.astype(int)]
        
        # Compute log_sum_exp for each sample
        max_logits = np.max(logits, axis=1)
        log_sum_exp = max_logits + np.log(np.sum(np.exp(logits - max_logits[:, np.newaxis]), axis=1))
        
        loss = log_sum_exp - target_logits
        
        if reduction == 'mean':
            return np.mean(loss)
        elif reduction == 'sum':
            return np.sum(loss)
        else:
            return loss

# Test classification losses
print("🧪 TESTING CLASSIFICATION LOSSES")
print("=" * 50)

# Generate test data
np.random.seed(42)
n_samples, n_classes = 5, 3

# Binary classification test
y_binary = np.array([0, 1, 1, 0, 1])
logits_binary = np.array([-2.0, 1.5, 0.5, -1.0, 3.0])
probs_binary = ClassificationLosses.sigmoid(logits_binary)

print("BINARY CLASSIFICATION TEST:")
print(f"True labels: {y_binary}")
print(f"Logits:      {logits_binary}")
print(f"Probabilities: {probs_binary}")

cls_losses = ClassificationLosses()

# Compare BCE implementations
bce_from_probs = cls_losses.binary_crossentropy(y_binary, probs_binary)
bce_from_logits = cls_losses.binary_crossentropy_with_logits(y_binary, logits_binary)

print(f"\nBinary Cross-Entropy:")
print(f"  From probabilities: {bce_from_probs:.6f}")
print(f"  From logits:        {bce_from_logits:.6f}")
print(f"  Difference:         {abs(bce_from_probs - bce_from_logits):.8f}")

# Test gradients
bce_grad = cls_losses.binary_crossentropy_gradient(y_binary, logits_binary)
print(f"  BCE gradient:       {bce_grad}")

# Multi-class classification test
print(f"\nMULTI-CLASS CLASSIFICATION TEST:")
y_categorical = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1], [1, 0, 0], [0, 1, 0]])  # One-hot
y_sparse = np.array([0, 1, 2, 0, 1])  # Integer labels
logits_multi = np.random.randn(n_samples, n_classes)
probs_multi = cls_losses.softmax(logits_multi)

print(f"True labels (one-hot): {y_categorical}")
print(f"True labels (sparse):  {y_sparse}")
print(f"Logits shape: {logits_multi.shape}")
print(f"Probabilities sum: {np.sum(probs_multi, axis=1)}")  # Should be ~1.0

# Compare CE implementations
ce_from_probs = cls_losses.categorical_crossentropy(y_categorical, probs_multi)
ce_from_logits = cls_losses.categorical_crossentropy_with_logits(y_categorical, logits_multi)
sparse_ce = cls_losses.sparse_categorical_crossentropy(y_sparse, logits_multi)

print(f"\nCategorical Cross-Entropy:")
print(f"  From probabilities: {ce_from_probs:.6f}")
print(f"  From logits:        {ce_from_logits:.6f}")
print(f"  Sparse version:     {sparse_ce:.6f}")

print(f"\n💡 INTERVIEW INSIGHT:")
print(f"   All three should give the same result!")
print(f"   Differences: {abs(ce_from_probs - ce_from_logits):.8f}, {abs(ce_from_logits - sparse_ce):.8f}")

# Test numerical stability
print(f"\n🔥 NUMERICAL STABILITY TEST:")
extreme_logits = np.array([-1000, 0, 1000])  # Extreme values
stable_softmax = cls_losses.softmax(extreme_logits)
print(f"Extreme logits: {extreme_logits}")
print(f"Stable softmax: {stable_softmax}")
print(f"Sum check: {np.sum(stable_softmax):.10f}")  # Should be 1.0

In [ ]:
# Add Focal Loss and Hinge Loss to ClassificationLosses
def focal_loss(y_true: np.ndarray, logits: np.ndarray, alpha: float = 0.25, 
               gamma: float = 2.0, reduction: str = 'mean') -> float:
    """
    Focal Loss for Addressing Class Imbalance
    
    📚 MATHEMATICAL FOUNDATION:
    FL(p_t) = -α_t (1 - p_t)^γ log(p_t)
    
    Where:
    • p_t = p if y=1, else (1-p)
    • α_t = α if y=1, else (1-α)
    
    🔍 GRADIENT (complex, involves chain rule):
    ∂FL/∂z = α_t * [(1-p_t)^γ (γp_t log(p_t) + p_t - y) - γ(1-p_t)^(γ-1) p_t log(p_t)]
    
    ⚠️ KEY PROPERTIES:
    • γ = 0 → Standard CE loss
    • γ > 0 → Down-weights easy examples
    • α balances positive/negative classes
    • Helps with extreme class imbalance
    
    🎯 BUSINESS APPLICATIONS:
    • Fraud detection (rare positives)
    • Medical diagnosis (rare diseases)
    • Object detection (background vs objects)
    
    Args:
        alpha: Weighting factor for positive class
        gamma: Focusing parameter (typically 2.0)
    """
    # Get probabilities
    probabilities = ClassificationLosses.sigmoid(logits)
    
    # Compute p_t (probability of true class)
    p_t = y_true * probabilities + (1 - y_true) * (1 - probabilities)
    
    # Compute α_t (class-specific weighting)
    alpha_t = y_true * alpha + (1 - y_true) * (1 - alpha)
    
    # Focal loss formula
    focal_weight = alpha_t * np.power(1 - p_t, gamma)
    ce_loss = -np.log(np.clip(p_t, EPSILON, 1 - EPSILON))
    loss = focal_weight * ce_loss
    
    if reduction == 'mean':
        return np.mean(loss)
    elif reduction == 'sum':
        return np.sum(loss)
    else:
        return loss

def hinge_loss(y_true: np.ndarray, raw_scores: np.ndarray, 
               margin: float = 1.0, reduction: str = 'mean') -> float:
    """
    Hinge Loss (SVM-style)
    
    📚 MATHEMATICAL FOUNDATION:
    L = max(0, margin - y * f(x))
    
    Where:
    • y ∈ {-1, +1} (NOT {0, 1}!)
    • f(x) = raw score (NOT probability)
    • margin = desired separation (typically 1.0)
    
    🔍 GRADIENT:
    ∂L/∂f = {
        -y  if y * f(x) < margin
        0   otherwise
    }
    
    ⚠️ PROPERTIES:
    • Non-probabilistic (raw scores)
    • Sparse gradients (many zeros)
    • Promotes large margin separation
    • Robust to outliers
    
    🎯 WHEN TO USE:
    • SVM-style classification
    • When you want margin maximization
    • Binary classification with clear separation
    
    Args:
        y_true: Labels in {-1, +1} format
        raw_scores: Raw model outputs (NOT probabilities)
        margin: Desired margin (typically 1.0)
    """
    # Ensure labels are in {-1, +1} format
    if np.any((y_true != -1) & (y_true != 1)):
        raise ValueError("Hinge loss requires labels in {-1, +1} format")
    
    # Hinge loss: max(0, margin - y * f(x))
    margin_violations = margin - y_true * raw_scores
    loss = np.maximum(0, margin_violations)
    
    if reduction == 'mean':
        return np.mean(loss)
    elif reduction == 'sum':
        return np.sum(loss)
    else:
        return loss

def hinge_gradient(y_true: np.ndarray, raw_scores: np.ndarray, margin: float = 1.0) -> np.ndarray:
    """Gradient of hinge loss w.r.t. raw scores"""
    margin_violations = margin - y_true * raw_scores
    # Gradient is -y where margin is violated, 0 otherwise
    return np.where(margin_violations > 0, -y_true, 0)

# Add methods to ClassificationLosses
ClassificationLosses.focal_loss = staticmethod(focal_loss)
ClassificationLosses.hinge_loss = staticmethod(hinge_loss)
ClassificationLosses.hinge_gradient = staticmethod(hinge_gradient)

# Demonstrate focal loss vs BCE
print("🎯 FOCAL LOSS vs BINARY CROSS-ENTROPY")
print("=" * 50)

# Create imbalanced dataset scenario
easy_examples = np.array([1, 1, 1, 1, 1])  # Easy positive examples
hard_examples = np.array([1, 1, 1, 1, 1])  # Hard positive examples

easy_logits = np.array([5, 4, 3, 4, 6])    # High confidence (easy)
hard_logits = np.array([0.1, -0.2, 0.3, -0.1, 0.2])  # Low confidence (hard)

print("EASY EXAMPLES (high confidence):")
print(f"Labels: {easy_examples}")
print(f"Logits: {easy_logits}")

easy_bce = cls_losses.binary_crossentropy_with_logits(easy_examples, easy_logits)
easy_focal = cls_losses.focal_loss(easy_examples, easy_logits, alpha=0.25, gamma=2.0)

print(f"BCE loss:   {easy_bce:.6f}")
print(f"Focal loss: {easy_focal:.6f}")
print(f"Focal reduction: {(easy_bce - easy_focal) / easy_bce * 100:.1f}%")

print(f"\nHARD EXAMPLES (low confidence):")
print(f"Labels: {hard_examples}")
print(f"Logits: {hard_logits}")

hard_bce = cls_losses.binary_crossentropy_with_logits(hard_examples, hard_logits)
hard_focal = cls_losses.focal_loss(hard_examples, hard_logits, alpha=0.25, gamma=2.0)

print(f"BCE loss:   {hard_bce:.6f}")
print(f"Focal loss: {hard_focal:.6f}")
print(f"Focal reduction: {(hard_bce - hard_focal) / hard_bce * 100:.1f}%")

print(f"\n💡 FOCAL LOSS INSIGHT:")
print(f"   Easy examples: {easy_focal/easy_bce:.3f}x of original BCE")
print(f"   Hard examples: {hard_focal/hard_bce:.3f}x of original BCE")
print(f"   → Focal loss focuses on hard examples!")

# Demonstrate hinge loss
print(f"\n⚔️ HINGE LOSS DEMONSTRATION")
print("=" * 40)

# Convert binary labels {0,1} to {-1,+1} for hinge loss
y_hinge = np.array([-1, 1, 1, -1, 1])  # Hinge format
raw_scores = np.array([-2, 3, 0.5, -0.5, 1.5])

print(f"Labels (hinge format): {y_hinge}")
print(f"Raw scores: {raw_scores}")

hinge_losses = cls_losses.hinge_loss(y_hinge, raw_scores, reduction='none')
hinge_grads = cls_losses.hinge_gradient(y_hinge, raw_scores)

print(f"Individual hinge losses: {hinge_losses}")
print(f"Hinge gradients: {hinge_grads}")
print(f"Mean hinge loss: {np.mean(hinge_losses):.4f}")

# Show margin violations
margins = y_hinge * raw_scores
print(f"\nMargin analysis (y * f(x)):")
for i, (y, score, margin) in enumerate(zip(y_hinge, raw_scores, margins)):
    status = "VIOLATES" if margin < 1.0 else "OK"
    print(f"  Sample {i}: y={y:2d}, f(x)={score:5.1f}, margin={margin:5.1f} [{status}]")

print(f"\n🎯 HINGE LOSS INSIGHT:")
print(f"   Only penalizes examples with margin < 1.0")
print(f"   Promotes large margin separation (SVM-style)")
print(f"   Sparse gradients → many parameters don't update")

## 4. Advanced Loss Functions

### 🎯 **Specialized Loss Functions for Modern Applications**

Beyond standard regression and classification, modern ML systems require specialized losses:

| **Loss Function** | **Application** | **Key Innovation** | **Amazon Use Cases** |
|-------------------|-----------------|-------------------|----------------------|
| **Triplet Loss** | Metric Learning | Embedding spaces | Product similarity, face recognition |
| **Contrastive Loss** | Siamese Networks | Pairwise similarity | Duplicate detection, recommendation |
| **Wasserstein Loss** | GANs | Distribution matching | Data generation, style transfer |
| **KL Divergence** | Probabilistic | Distribution comparison | VAEs, knowledge distillation |
| **Label Smoothing** | Regularization | Confidence calibration | Overconfident model prevention |

### 🚨 **Advanced Challenges:**
1. **Hard negative mining**: Selecting informative training examples
2. **Margin tuning**: Balancing similarity/dissimilarity
3. **Batch composition**: Ensuring diverse training batches
4. **Computational efficiency**: Scaling to large embedding spaces

### 🎯 **Interview Deep Dive:**
- When standard losses fail and why
- How to design custom losses for business problems
- Numerical challenges in metric learning
- Connection between loss choice and model architecture

In [ ]:
class AdvancedLosses:
    """
    Advanced loss functions for specialized applications.
    
    🎯 INTERVIEW FOCUS: Metric learning and distribution matching
    """
    
    @staticmethod
    def triplet_loss(anchor: np.ndarray, positive: np.ndarray, negative: np.ndarray,
                     margin: float = 1.0, distance_metric: str = 'l2', reduction: str = 'mean') -> float:
        """
        Triplet Loss for Metric Learning
        
        📚 MATHEMATICAL FOUNDATION:
        L = max(0, d(a,p) - d(a,n) + margin)
        
        Where:
        • d(a,p) = distance(anchor, positive)
        • d(a,n) = distance(anchor, negative)
        • margin = minimum separation desired
        
        🔍 GRADIENT (L2 distance):
        ∂L/∂a = 2(a-p) - 2(a-n) = 2(p-n)  [if loss > 0]
        ∂L/∂p = 2(p-a)                    [if loss > 0]
        ∂L/∂n = 2(n-a)                    [if loss > 0]
        
        ⚠️ PROPERTIES:
        • Forces d(a,p) + margin < d(a,n)
        • Creates embedding space with semantic structure
        • Requires careful triplet mining
        • Sensitive to margin choice
        
        🎯 AMAZON APPLICATIONS:
        • Product similarity (same product, different sellers)
        • Customer embeddings for recommendation
        • Content-based similarity matching
        
        Args:
            anchor: Reference embeddings
            positive: Similar embeddings
            negative: Dissimilar embeddings
            margin: Minimum separation between pos/neg
            distance_metric: 'l2' or 'cosine'
        """
        if distance_metric == 'l2':
            # L2 (Euclidean) distance
            pos_dist = np.sum((anchor - positive)**2, axis=-1)
            neg_dist = np.sum((anchor - negative)**2, axis=-1)
        elif distance_metric == 'cosine':
            # Cosine distance = 1 - cosine_similarity
            def cosine_distance(x, y):
                dot_product = np.sum(x * y, axis=-1)
                norm_x = np.linalg.norm(x, axis=-1)
                norm_y = np.linalg.norm(y, axis=-1)
                cosine_sim = dot_product / (norm_x * norm_y + EPSILON)
                return 1 - cosine_sim
            
            pos_dist = cosine_distance(anchor, positive)
            neg_dist = cosine_distance(anchor, negative)
        else:
            raise ValueError("distance_metric must be 'l2' or 'cosine'")
        
        # Triplet loss: max(0, d_pos - d_neg + margin)
        loss = np.maximum(0, pos_dist - neg_dist + margin)
        
        if reduction == 'mean':
            return np.mean(loss)
        elif reduction == 'sum':
            return np.sum(loss)
        else:
            return loss
    
    @staticmethod
    def contrastive_loss(embedding1: np.ndarray, embedding2: np.ndarray, labels: np.ndarray,
                        margin: float = 1.0, reduction: str = 'mean') -> float:
        """
        Contrastive Loss for Siamese Networks
        
        📚 MATHEMATICAL FOUNDATION:
        L = (1-y) * ½d² + y * ½max(0, margin - d)²
        
        Where:
        • y = 1 if dissimilar pair, 0 if similar pair
        • d = ||f(x1) - f(x2)||₂ (L2 distance)
        
        🔍 GRADIENT:
        ∂L/∂f(x1) = {
            (1-y) * (f(x1) - f(x2))           [similar pairs]
            y * max(0, 1 - d/margin) * (f(x1) - f(x2))/d    [dissimilar pairs]
        }
        
        ⚠️ PROPERTIES:
        • Pulls similar pairs together (minimize distance)
        • Pushes dissimilar pairs apart (up to margin)
        • Simpler than triplet loss (only pairs needed)
        • Good for verification tasks
        
        🎯 AMAZON APPLICATIONS:
        • Duplicate product detection
        • User account verification
        • Content similarity matching
        
        Args:
            embedding1: First set of embeddings
            embedding2: Second set of embeddings
            labels: 0 for similar pairs, 1 for dissimilar pairs
            margin: Maximum distance for dissimilar pairs
        """
        # Compute L2 distance
        distances = np.linalg.norm(embedding1 - embedding2, axis=-1)
        
        # Similar pairs: minimize distance
        similar_loss = (1 - labels) * 0.5 * distances**2
        
        # Dissimilar pairs: maximize distance up to margin
        dissimilar_loss = labels * 0.5 * np.maximum(0, margin - distances)**2
        
        loss = similar_loss + dissimilar_loss
        
        if reduction == 'mean':
            return np.mean(loss)
        elif reduction == 'sum':
            return np.sum(loss)
        else:
            return loss
    
    @staticmethod
    def kl_divergence(p: np.ndarray, q: np.ndarray, epsilon: float = EPSILON) -> float:
        """
        Kullback-Leibler Divergence
        
        📚 MATHEMATICAL FOUNDATION:
        KL(P||Q) = Σ p(x) log(p(x)/q(x))
        
        🔍 PROPERTIES:
        • Measures how P differs from Q
        • Non-symmetric: KL(P||Q) ≠ KL(Q||P)
        • Always non-negative
        • Zero iff P = Q
        
        ⚠️ NUMERICAL ISSUES:
        • log(0) = -∞ → clip probabilities
        • p(x) > 0 but q(x) = 0 → infinite divergence
        
        🎯 APPLICATIONS:
        • VAE loss (regularization term)
        • Knowledge distillation (teacher-student)
        • Distribution matching
        • Information theory measures
        
        Args:
            p: True distribution (probabilities)
            q: Approximate distribution (probabilities)
        """
        # Clip to prevent log(0)
        p_clipped = np.clip(p, epsilon, 1)
        q_clipped = np.clip(q, epsilon, 1)
        
        # KL divergence formula
        return np.sum(p_clipped * np.log(p_clipped / q_clipped))
    
    @staticmethod
    def jensen_shannon_divergence(p: np.ndarray, q: np.ndarray) -> float:
        """
        Jensen-Shannon Divergence (Symmetric version of KL)
        
        📚 MATHEMATICAL FOUNDATION:
        JS(P||Q) = ½KL(P||M) + ½KL(Q||M)
        Where M = ½(P + Q)
        
        ⚠️ PROPERTIES:
        • Symmetric: JS(P||Q) = JS(Q||P)
        • Bounded: 0 ≤ JS ≤ log(2)
        • Smooth and well-behaved
        """
        # Compute mixture distribution
        m = 0.5 * (p + q)
        
        # Symmetric KL divergence
        return 0.5 * AdvancedLosses.kl_divergence(p, m) + 0.5 * AdvancedLosses.kl_divergence(q, m)
    
    @staticmethod
    def label_smoothing_crossentropy(y_true: np.ndarray, logits: np.ndarray, 
                                   smoothing: float = 0.1, reduction: str = 'mean') -> float:
        """
        Label Smoothing Cross-Entropy
        
        📚 MATHEMATICAL FOUNDATION:
        Instead of hard labels [0, 0, 1, 0], use soft labels:
        y_smooth = (1 - ε) * y_true + ε / K
        
        Where:
        • ε = smoothing parameter
        • K = number of classes
        
        🔍 BENEFITS:
        • Prevents overconfident predictions
        • Improves model calibration
        • Acts as regularization
        • Better generalization
        
        🎯 WHEN TO USE:
        • Large models prone to overfitting
        • When prediction confidence matters
        • Classification with many classes
        
        Args:
            smoothing: Amount of label smoothing (typically 0.1)
        """
        num_classes = logits.shape[-1]
        
        # Create smoothed labels
        smoothed_labels = y_true * (1 - smoothing) + smoothing / num_classes
        
        # Use standard categorical CE with smoothed labels
        return ClassificationLosses.categorical_crossentropy_with_logits(
            smoothed_labels, logits, reduction=reduction)

# Test advanced losses
print("🔬 TESTING ADVANCED LOSS FUNCTIONS")
print("=" * 50)

adv_losses = AdvancedLosses()

# Generate test embeddings for triplet loss
np.random.seed(42)
embedding_dim = 128
anchor = np.random.randn(5, embedding_dim)
positive = anchor + 0.1 * np.random.randn(5, embedding_dim)  # Similar to anchor
negative = np.random.randn(5, embedding_dim)  # Random (likely dissimilar)

# Test triplet loss
triplet_l2 = adv_losses.triplet_loss(anchor, positive, negative, margin=1.0, distance_metric='l2')
triplet_cosine = adv_losses.triplet_loss(anchor, positive, negative, margin=0.2, distance_metric='cosine')

print("TRIPLET LOSS TEST:")
print(f"L2 distance triplet loss:     {triplet_l2:.6f}")
print(f"Cosine distance triplet loss: {triplet_cosine:.6f}")

# Compute individual distances for analysis
pos_dists_l2 = np.sum((anchor - positive)**2, axis=1)
neg_dists_l2 = np.sum((anchor - negative)**2, axis=1)

print(f"\nDistance analysis (L2):")
print(f"Positive distances (should be small): {pos_dists_l2}")
print(f"Negative distances (should be large): {neg_dists_l2}")
print(f"Margins (neg - pos): {neg_dists_l2 - pos_dists_l2}")

# Test contrastive loss
labels_contrastive = np.array([0, 0, 1, 1, 0])  # 0=similar, 1=dissimilar
contrastive_loss_val = adv_losses.contrastive_loss(
    anchor, positive, labels_contrastive, margin=2.0)

print(f"\nCONTRASTIVE LOSS TEST:")
print(f"Contrastive loss: {contrastive_loss_val:.6f}")
print(f"Labels (0=similar, 1=dissimilar): {labels_contrastive}")

# Test KL divergence
p_dist = np.array([0.7, 0.2, 0.1])  # True distribution
q_dist = np.array([0.6, 0.3, 0.1])  # Approximate distribution

kl_pq = adv_losses.kl_divergence(p_dist, q_dist)
kl_qp = adv_losses.kl_divergence(q_dist, p_dist)
js_div = adv_losses.jensen_shannon_divergence(p_dist, q_dist)

print(f"\nDIVERGENCE TEST:")
print(f"P distribution: {p_dist}")
print(f"Q distribution: {q_dist}")
print(f"KL(P||Q): {kl_pq:.6f}")
print(f"KL(Q||P): {kl_qp:.6f}")
print(f"JS(P||Q): {js_div:.6f} (symmetric)")

# Test label smoothing
y_one_hot = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])
test_logits = np.random.randn(3, 3)

normal_ce = ClassificationLosses.categorical_crossentropy_with_logits(y_one_hot, test_logits)
smoothed_ce = adv_losses.label_smoothing_crossentropy(y_one_hot, test_logits, smoothing=0.1)

print(f"\nLABEL SMOOTHING TEST:")
print(f"Normal CE loss:     {normal_ce:.6f}")
print(f"Label smoothed CE:  {smoothed_ce:.6f}")
print(f"Smoothing effect:   {(smoothed_ce - normal_ce):.6f}")

print(f"\n🎯 ADVANCED LOSS INSIGHTS:")
print(f"• Triplet loss learns embedding spaces with semantic structure")
print(f"• Contrastive loss is simpler but requires good pair mining")
print(f"• KL divergence is fundamental to many modern architectures")
print(f"• Label smoothing prevents overconfident predictions")

In [ ]:
# Comprehensive Loss Function Visualization and Comparison
def create_comprehensive_loss_visualization():
    """
    Create comprehensive visualizations comparing all loss functions
    """
    fig, axes = plt.subplots(3, 3, figsize=(20, 15))
    
    # 1. Regression Loss Comparison
    ax = axes[0, 0]
    errors = np.linspace(-3, 3, 1000)
    
    reg_losses = RegressionLosses()
    mse_vals = [reg_losses.mse_loss(np.array([0]), np.array([e]), reduction='none')[0] for e in errors]
    mae_vals = [reg_losses.mae_loss(np.array([0]), np.array([e]), reduction='none')[0] for e in errors]
    huber_vals = [reg_losses.huber_loss(np.array([0]), np.array([e]), delta=1.0, reduction='none')[0] for e in errors]
    
    ax.plot(errors, mse_vals, 'b-', linewidth=2, label='MSE (L2)')
    ax.plot(errors, mae_vals, 'r-', linewidth=2, label='MAE (L1)')
    ax.plot(errors, huber_vals, 'g-', linewidth=2, label='Huber')
    ax.set_xlabel('Error (ŷ - y)')
    ax.set_ylabel('Loss Value')
    ax.set_title('Regression Losses', fontweight='bold', fontsize=14)
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # 2. Classification Loss Comparison
    ax = axes[0, 1]
    probs = np.linspace(0.001, 0.999, 1000)
    
    # Binary CE for positive class (y=1)
    bce_pos = -np.log(probs)
    bce_neg = -np.log(1 - probs)
    
    ax.plot(probs, bce_pos, 'b-', linewidth=2, label='BCE (y=1)')
    ax.plot(probs, bce_neg, 'r-', linewidth=2, label='BCE (y=0)')
    ax.set_xlabel('Predicted Probability')
    ax.set_ylabel('Loss Value')
    ax.set_title('Binary Cross-Entropy', fontweight='bold', fontsize=14)
    ax.legend()
    ax.grid(True, alpha=0.3)
    ax.set_yscale('log')
    
    # 3. Focal Loss vs BCE
    ax = axes[0, 2]
    # Focal loss for different gamma values
    for gamma in [0, 1, 2, 5]:
        focal_vals = -(1 - probs)**gamma * np.log(probs)
        label = f'Focal (γ={gamma})' if gamma > 0 else 'BCE (γ=0)'
        ax.plot(probs, focal_vals, linewidth=2, label=label)
    
    ax.set_xlabel('Predicted Probability (y=1)')
    ax.set_ylabel('Loss Value')
    ax.set_title('Focal Loss Effect', fontweight='bold', fontsize=14)
    ax.legend()
    ax.grid(True, alpha=0.3)
    ax.set_yscale('log')
    
    # 4. Hinge Loss
    ax = axes[1, 0]
    scores = np.linspace(-3, 3, 1000)
    hinge_pos = np.maximum(0, 1 - scores)  # y = +1
    hinge_neg = np.maximum(0, 1 + scores)  # y = -1
    
    ax.plot(scores, hinge_pos, 'b-', linewidth=2, label='Hinge (y=+1)')
    ax.plot(scores, hinge_neg, 'r-', linewidth=2, label='Hinge (y=-1)')
    ax.axvline(x=1, color='b', linestyle='--', alpha=0.7, label='Margin (y=+1)')
    ax.axvline(x=-1, color='r', linestyle='--', alpha=0.7, label='Margin (y=-1)')
    ax.set_xlabel('Raw Score f(x)')
    ax.set_ylabel('Loss Value')
    ax.set_title('Hinge Loss (SVM)', fontweight='bold', fontsize=14)
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # 5. Quantile Loss
    ax = axes[1, 1]
    for tau in [0.1, 0.5, 0.9]:
        quantile_vals = []
        for e in errors:
            if e >= 0:  # Under-prediction
                loss = tau * e
            else:  # Over-prediction
                loss = (tau - 1) * e
            quantile_vals.append(loss)
        ax.plot(errors, quantile_vals, linewidth=2, label=f'τ={tau}')
    
    ax.set_xlabel('Error (y - ŷ)')
    ax.set_ylabel('Loss Value')
    ax.set_title('Quantile Loss', fontweight='bold', fontsize=14)
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # 6. Triplet Loss Illustration
    ax = axes[1, 2]
    # Simulate triplet loss landscape
    distances = np.linspace(0, 3, 100)
    margin = 1.0
    
    # For different d(a,n) values
    for d_an in [0.5, 1.0, 1.5, 2.0]:
        triplet_vals = np.maximum(0, distances - d_an + margin)
        ax.plot(distances, triplet_vals, linewidth=2, label=f'd(a,n)={d_an}')
    
    ax.set_xlabel('d(a,p) - Distance to Positive')
    ax.set_ylabel('Triplet Loss')
    ax.set_title('Triplet Loss Landscape', fontweight='bold', fontsize=14)
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # 7. KL Divergence
    ax = axes[2, 0]
    p_true = 0.7  # True probability
    q_range = np.linspace(0.001, 0.999, 1000)
    
    kl_vals = p_true * np.log(p_true / q_range) + (1 - p_true) * np.log((1 - p_true) / (1 - q_range))
    
    ax.plot(q_range, kl_vals, 'purple', linewidth=2)
    ax.axvline(x=p_true, color='red', linestyle='--', linewidth=2, label=f'True p={p_true}')
    ax.set_xlabel('Predicted Probability q')
    ax.set_ylabel('KL(p||q)')
    ax.set_title('KL Divergence', fontweight='bold', fontsize=14)
    ax.legend()
    ax.grid(True, alpha=0.3)
    ax.set_yscale('log')
    
    # 8. Gradient Comparison
    ax = axes[2, 1]
    # Compare gradients of different loss functions
    mse_grads = errors  # MSE gradient is linear
    mae_grads = np.sign(errors)  # MAE gradient is constant
    huber_grads = np.where(np.abs(errors) <= 1, errors, np.sign(errors))  # Huber gradient
    
    ax.plot(errors, mse_grads, 'b-', linewidth=2, label='MSE Gradient')
    ax.plot(errors, mae_grads, 'r-', linewidth=2, label='MAE Gradient')
    ax.plot(errors, huber_grads, 'g-', linewidth=2, label='Huber Gradient')
    ax.set_xlabel('Error (ŷ - y)')
    ax.set_ylabel('Gradient Value')
    ax.set_title('Loss Function Gradients', fontweight='bold', fontsize=14)
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # 9. Business Impact Summary
    ax = axes[2, 2]
    ax.axis('off')
    
    business_text = '''
    BUSINESS IMPACT OF LOSS CHOICE
    
    📊 Regression:
    • MSE: Minimizes variance (Gaussian noise)
    • MAE: Minimizes median error (robust)
    • Huber: Balanced approach
    • Quantile: Risk assessment, SLA targets
    
    🎯 Classification:
    • BCE: Standard binary classification
    • Focal: Addresses class imbalance
    • Hinge: Maximum margin separation
    
    🔬 Advanced:
    • Triplet: Learning similarity metrics
    • KL: Distribution matching, VAEs
    • Label Smoothing: Calibration
    
    ⚠️ KEY INSIGHT:
    Loss function choice directly impacts:
    • Model behavior and robustness
    • Training dynamics and convergence
    • Business metrics alignment
    • Prediction confidence and calibration
    '''
    
    ax.text(0.05, 0.95, business_text, transform=ax.transAxes, fontsize=11,
            verticalalignment='top', fontfamily='monospace',
            bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.8))
    
    plt.tight_layout()
    plt.suptitle('Complete Loss Function Reference', fontsize=16, fontweight='bold', y=0.98)
    plt.show()

# Create comprehensive visualization
create_comprehensive_loss_visualization()

# Final performance comparison
def benchmark_loss_computations():
    """Benchmark computational performance of different losses"""
    import time
    
    print("⚡ COMPUTATIONAL PERFORMANCE BENCHMARK")
    print("=" * 50)
    
    # Generate test data
    n_samples = 10000
    n_classes = 100
    
    np.random.seed(42)
    y_true_reg = np.random.randn(n_samples)
    y_pred_reg = np.random.randn(n_samples)
    
    y_true_clf = np.random.randint(0, 2, n_samples)
    logits_clf = np.random.randn(n_samples)
    
    y_true_multi = np.eye(n_classes)[np.random.randint(0, n_classes, n_samples)]
    logits_multi = np.random.randn(n_samples, n_classes)
    
    # Benchmark regression losses
    reg_losses = RegressionLosses()
    
    start_time = time.time()
    for _ in range(100):
        reg_losses.mse_loss(y_true_reg, y_pred_reg)
    mse_time = time.time() - start_time
    
    start_time = time.time()
    for _ in range(100):
        reg_losses.mae_loss(y_true_reg, y_pred_reg)
    mae_time = time.time() - start_time
    
    start_time = time.time()
    for _ in range(100):
        reg_losses.huber_loss(y_true_reg, y_pred_reg)
    huber_time = time.time() - start_time
    
    # Benchmark classification losses
    cls_losses = ClassificationLosses()
    
    start_time = time.time()
    for _ in range(100):
        cls_losses.binary_crossentropy_with_logits(y_true_clf, logits_clf)
    bce_time = time.time() - start_time
    
    start_time = time.time()
    for _ in range(100):
        cls_losses.categorical_crossentropy_with_logits(y_true_multi, logits_multi)
    ce_time = time.time() - start_time
    
    print(f"REGRESSION LOSSES (100 iterations, {n_samples} samples):")
    print(f"  MSE:   {mse_time:.4f}s")
    print(f"  MAE:   {mae_time:.4f}s")
    print(f"  Huber: {huber_time:.4f}s")
    
    print(f"\nCLASSIFICATION LOSSES (100 iterations):")
    print(f"  Binary CE:     {bce_time:.4f}s")
    print(f"  Categorical CE: {ce_time:.4f}s")
    
    print(f"\n💡 PERFORMANCE INSIGHTS:")
    print(f"• MSE is fastest (simple arithmetic)")
    print(f"• MAE has conditional logic overhead")
    print(f"• Huber combines both → moderate speed")
    print(f"• Logit-based losses avoid sigmoid computation")

# Run benchmark
benchmark_loss_computations()

print("\n🎓 LOSS FUNCTION MASTERY COMPLETE!")
print("=" * 50)
print("✅ Mathematical foundations understood")
print("✅ Custom implementations working")
print("✅ Numerical stability handled")
print("✅ Gradient derivations covered")
print("✅ Business applications clear")
print("✅ Performance characteristics known")
print("\n🚀 You're ready for Amazon Applied Scientist interviews!")

## 5. Amazon Interview Preparation Summary

### 🎯 **Quick Reference for Interviews**

#### **Essential Loss Function Facts:**

| **Loss** | **Use Case** | **Key Property** | **Gradient** | **When It Fails** |
|----------|--------------|------------------|--------------|-------------------|
| **MSE** | Regression | Quadratic penalty | Linear growth | Outliers present |
| **MAE** | Robust regression | Linear penalty | Constant | Need smooth gradients |
| **Huber** | Balanced regression | Hybrid L1/L2 | Clipped linear | Need pure L1/L2 |
| **BCE** | Binary classification | Probabilistic | p - y | Class imbalance |
| **Focal** | Imbalanced classification | Down-weights easy | Complex | Balanced datasets |
| **Hinge** | SVM-style | Large margin | Sparse | Need probabilities |
| **Triplet** | Metric learning | Embedding structure | Distance-based | Poor triplet mining |

### 🚨 **Critical Interview Topics**

#### **1. Mathematical Derivations** (Must Know)
```python
# Be ready to derive these on whiteboard:

# MSE gradient:
# L = ½(y - ŷ)²
# ∂L/∂ŷ = -(y - ŷ) = (ŷ - y)

# BCE gradient (with sigmoid):
# L = -[y log σ(z) + (1-y) log(1-σ(z))]
# ∂L/∂z = σ(z) - y

# Cross-entropy gradient (with softmax):
# L = -Σ y_i log(softmax_i(z))
# ∂L/∂z_i = softmax_i(z) - y_i
```

#### **2. Numerical Stability** (Critical)
```python
# Always mention these in interviews:

# 1. Sigmoid overflow prevention
def safe_sigmoid(z):
    return np.where(z >= 0, 1/(1 + np.exp(-z)), np.exp(z)/(1 + np.exp(z)))

# 2. Log-sum-exp trick
def stable_softmax(logits):
    shifted = logits - np.max(logits, axis=-1, keepdims=True)
    return np.exp(shifted) / np.sum(np.exp(shifted), axis=-1, keepdims=True)

# 3. Probability clipping
def safe_log(p, epsilon=1e-15):
    return np.log(np.clip(p, epsilon, 1 - epsilon))
```

#### **3. Business Alignment** (Amazon Specific)
- **Customer obsession**: How does loss choice impact user experience?
- **Ownership**: Why did you choose this loss over alternatives?
- **Dive deep**: What are the mathematical trade-offs?
- **Bias for action**: Quick prototyping vs. optimal solution

### 🎯 **Sample Interview Questions & Answers**

#### **Q: "Why would you use Huber loss instead of MSE?"**
**A:** "Huber loss provides the best of both worlds. For small errors (≤δ), it behaves like MSE with smooth gradients for stable optimization. For large errors (>δ), it behaves like MAE with linear growth, making it robust to outliers. This is crucial in production systems where data quality isn't guaranteed. I'd choose δ based on the expected noise level in the data."

#### **Q: "Implement focal loss and explain when to use it."**
**A:** "Focal loss addresses the class imbalance problem that's common in real-world applications like fraud detection or medical diagnosis. The key insight is down-weighting easy examples with the term (1-p_t)^γ. When γ=0, it reduces to standard BCE. As γ increases, the model focuses more on hard examples. I typically start with γ=2 and α=0.25, then tune based on validation metrics that matter for the business problem."

#### **Q: "How do you ensure numerical stability in loss computations?"**
**A:** "Three main strategies: 1) Work in log-space when possible (e.g., log-softmax instead of softmax), 2) Use the log-sum-exp trick to prevent overflow, 3) Clip probabilities to [ε, 1-ε] to prevent log(0). For sigmoid, I use the stable implementation that separates positive and negative logits. These aren't just theoretical concerns - in production with extreme inputs, naive implementations will produce NaN gradients and crash training."

### 🔧 **Debugging Loss Functions**

#### **Common Issues & Solutions:**
1. **NaN losses**: Check for log(0), clip probabilities
2. **Exploding gradients**: Verify numerical stability, check learning rate
3. **Vanishing gradients**: Consider focal loss, check activation saturation
4. **Slow convergence**: Examine loss landscape, try different formulations
5. **Poor calibration**: Consider label smoothing or temperature scaling

### 🚀 **Advanced Topics for Senior Roles**

1. **Custom loss design**: How to create domain-specific losses
2. **Loss landscape analysis**: Understanding optimization dynamics
3. **Multi-task learning**: Balancing multiple loss components
4. **Adversarial robustness**: Losses that promote model stability
5. **Fairness constraints**: Incorporating bias mitigation into loss functions

### 📚 **Key Takeaways for Amazon**

1. **Customer impact**: Always connect loss choice to business metrics
2. **Scalability**: Consider computational cost at Amazon scale
3. **Robustness**: Production systems need stable, predictable behavior
4. **Interpretability**: Stakeholders need to understand model decisions
5. **Continuous improvement**: Loss functions evolve with business needs

---

**🎯 Final Interview Tip**: Practice explaining loss functions in simple terms to non-technical stakeholders. Amazon values leaders who can communicate complex concepts clearly and connect technical decisions to customer outcomes.